In [1]:
import numpy as np

#Before training,the mean must be substract
def jbtrain(trainingset,label):
    # the total num of image
    m=len(label)
    
    # the dim of features
    n=trainingset.shape[1]
#     label = label.reshape(1, len(label))   
    label = np.squeeze(np.asarray(label))
    
    classes, labels = np.unique(label, return_inverse=True)
    print('classes.size: {}'.format(len(classes)))
    print('index: {}'.format(labels))
    
    print('feature shape {}, dim of feature {}, label shape {}, unique labels {}'.format(trainingset.shape, n, label.shape, len(classes)))
    
    # filter the complicate label,for count the total people num
#     classes,labels=np.unique(label,return_inverse=True)
    # the total people num
    nc=len(classes)
  
    Sw=np.zeros([n,n])
    # save each people items
    cur={}
    withinCount=0
    # record the count of each people
    numberBuff=np.zeros(1000)
    
    for i in range(nc):
        
        # get the item of i
        cur[i]=trainingset[labels==i]     ## ！！！
        if cur[i].shape[0]>1:
            withinCount=withinCount+cur[i].shape[0]
        if numberBuff[cur[i].shape[0]] == 0 :
            numberBuff[cur[i].shape[0]] = 1
#         print('current label: {}/{}'.format(i, len(cur[i])))
    u=np.zeros([n,nc])
    ep=np.zeros([n,withinCount])
    nowp=0
    for i in range(nc):
        # the mean of cur[i]
        u[:,i]=np.mean(cur[i],0)
        b=u[:,i].reshape(n,1)
        if cur[i].shape[0]>1:
            ep[:,nowp:nowp+cur[i].shape[0]]=cur[i].T-b
            nowp=nowp+cur[i].shape[0]
    Su=np.cov(u.T,rowvar=0)
    Sw=np.cov(ep.T,rowvar=0)
    oldSw=Sw
    SuFG={}
    SwG={}
    for l in range(500):
        F=np.linalg.pinv(Sw)
        u=np.zeros([n,nc])
        ep=np.zeros([n,withinCount])
        nowp=0
        for g in range(1000):
            if numberBuff[g]==1:
                #G = −(mS μ + S ε )−1*Su*Sw−1
                G=-np.dot(np.dot(np.linalg.pinv(g*Su+Sw),Su),F)
                #Su*(F+g*G) for u
                SuFG[g]=np.dot(Su,(F+g*G))
                #Sw*G for e
                SwG[g]=np.dot(Sw,G)
        for i in range(nc):
            nnc=cur[i].shape[0]
            #formula 7 in suppl_760
            u[:,i]=np.sum(np.dot(SuFG[nnc],cur[i].T),1).ravel()     
            #formula 8 in suppl_760
            ep[:,nowp:nowp+cur[i].shape[0]]=cur[i].T+np.sum(np.dot(SwG[nnc],cur[i].T),1).reshape(n,1)
            nowp=nowp+nnc
        Su=np.cov(u.T,rowvar=0)
        Sw=np.cov(ep.T,rowvar=0)
        print(l,np.linalg.norm(Sw-oldSw)/np.linalg.norm(Sw))
### Future work: need to consider below
#         if np.linalg.norm(Sw-oldSw)/np.linalg.norm(Sw)<1e-6:
#             break;
        oldSw=Sw
    F=np.linalg.pinv(Sw)
    G=-np.dot(np.dot(np.linalg.pinv(2*Su+Sw),Su),F)
    A=np.linalg.pinv(Su+Sw)-(F+G)
    return A,G

#ratio of similar,the threshold we always choose in (-1,-2)
def jbverify(A,G,x1,x2):
#     x1.shape=(-1,1)
#     x2.shape=(-1,1)
    ratio=np.dot(np.dot(np.transpose(x1),A),x1)+np.dot(np.dot(np.transpose(x2),A),x2)-2*np.dot(np.dot(np.transpose(x1),G),x2)
    return ratio

In [2]:
###   load PCA model and train/label dataset 
###


import torch
import time
from sklearn import decomposition


feature_extract_tensor_file = '../checkpoints/feature_extract_full.pth'
feature_extract_after_PCA_tensor_file = '../checkpoints/feature_extract_after_PCA_full.pth'
feature_extract_without_PCA_tensor_file = '../checkpoints/feature_extract_without_PCA_full.pth'
jointBayesianModel_file = '../checkpoints/jointBayesianModel.pth'


PCA = True

if PCA == True:
    featureAfterPCAList = torch.load(feature_extract_after_PCA_tensor_file)
    print(featureAfterPCAList["features"][0])
    print(featureAfterPCAList["labels"][0])
    train_feat = featureAfterPCAList["features"]
    train_label = featureAfterPCAList["labels"]
    pca_model = featureAfterPCAList["PCAfit"]
else:
    featureWithoutPCAList = torch.load(feature_extract_without_PCA_tensor_file)
    print(featureWithoutPCAList["features"][0])
    print(featureWithoutPCAList["labels"][0]) 
    train_feat = featureWithoutPCAList["features"]
    train_label = featureWithoutPCAList["labels"]
    

[-3.0541306  -1.5055318  -2.897702   -5.793125    0.2771138  -4.4020348
  7.282463   -1.5974728  -5.74746     1.0986913   0.833211   -2.4517314
 -0.10165278  0.4647987  -2.054618   -1.8822994   1.4176805   0.09736018
 -0.8606803  -0.11187399]
[[159]]


In [4]:
###################################
#   training joint bayesian and save the model
###################################

import torch
import time

print('Training...')
t1 = time.time()
A, G = jbtrain(train_feat, train_label)
t2 = time.time()
print('Finished in %f seconds' % (t2-t1))

jointBayesianModel = {
    "A": A,
    "G": G
}

torch.save(jointBayesianModel, jointBayesianModel_file)
print("done")


classes.size: 431
index: [159 395 339 ...  38 427 312]
feature shape (16016, 20), dim of feature 20, label shape (16016,), unique labels 431
0 0.0004459704573004969
1 4.441752109732755e-05
2 2.8616810576885364e-06
3 1.9854970317244426e-07
4 1.4382000173598067e-08
5 1.0640421804675028e-09
6 7.956487981108916e-11
7 5.982788037735494e-12
8 4.5125428559680874e-13
9 3.407310323570328e-14
10 2.567088906271378e-15
11 2.642893848727321e-16
12 1.935861720585969e-16
13 2.156448922050628e-16
14 2.7947010976472984e-16
15 2.1703406213831347e-16
16 2.154123651203211e-16
17 2.3338132928548964e-16
18 2.427605402528534e-16
19 1.881728267523618e-16
20 2.4501361724281054e-16
21 2.3147141378658607e-16
22 1.8434169652300232e-16
23 1.4951466688828406e-16
24 1.9585240582561958e-16
25 2.0593561608539439e-16
26 1.8173082756734665e-16
27 1.6724810945807867e-16
28 2.3261258063327483e-16
29 2.361878981861874e-16
30 3.4109435701357026e-16
31 2.091580036116669e-16
32 2.0717882782993379e-16
33 2.25995309741949e-16
3

In [5]:
###################################
#   load joint bayesian model
###################################

import torch
import time
feature_extract_after_PCA_tensor_file = '../checkpoints/feature_extract_after_PCA_full.pth'
jointBayesianModel_file = '../checkpoints/jointBayesianModel.pth'

jointBayesianModel = torch.load(jointBayesianModel_file)
A = jointBayesianModel["A"]
G = jointBayesianModel["G"]
print('A Shape {} len {} ,G shape {} len {}'.format(A.shape, len(A), G.shape, len(G)))

A Shape (20, 20) len 20 ,G shape (20, 20) len 20


In [6]:
import numpy as np
import pandas as pd
import gc
import csv

import warnings
warnings.filterwarnings('ignore')

import os
import datetime
import time
import glob
import os.path as osp
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline


import torch
import torchvision
from torchvision import datasets, models
import torchvision.transforms as transforms
from torch.utils import data as D
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from tensorboardX import SummaryWriter

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
print("total GPU: %d" %(torch.cuda.device_count()))

class DefaultConfig(object):
    env = 'default' # visdom 环境
    path = '../data/compcars/data/cropped_image/'   #../data/compcars/data/image
    labelPath = '../data/compcars/data/label/'
    miscPath = '../data/compcars/data/misc/'
    modleNameFile = '../data/compcars/data/misc/modle_names.csv'
    makeNameFile = '../data/compcars/data/misc/make_names.csv'
    trainFilename = '../data/compcars/data/train_test_split/classification/train.txt'
    testFilename = '../data/compcars/data/train_test_split/classification/test.txt'
    verificationEasyFilename = '../data/compcars/data/train_test_split/verification/verification_pairs_easy.txt'
    verificationMediumFilename = '../data/compcars/data/train_test_split/verification/verification_pairs_medium.txt'
    verificationHardFilename = '../data/compcars/data/train_test_split/verification/verification_pairs_hard.txt'
    log_dir = '../log/'
    num_workers = 8
    num_epoches = 100
    num_classes = 431
    lr = 0.1
    lr_decay = 0.95
    max_epoch = 200000
    weight_decay = 1e-4
    load_model_path = '../checkpoints/train.pth'
    previous_loss = 1e100
    modle_csv_file = 'modle_names.csv'
    jointBayesianThreshold = -100

opt = DefaultConfig()
logger = SummaryWriter(opt.log_dir, flush_secs=2)


cuda:0
total GPU: 2


In [7]:
###############################
# load previous save model
###############################

# Load the model 
continue_model_inception3 = torchvision.models.inception_v3(pretrained=None)
net = continue_model_inception3

# # # modify output classes
num_ftrs = net.fc.in_features
net.fc = torch.nn.Linear(num_ftrs, opt.num_classes)    #num_ftrs = 2048

criterion = torch.nn.CrossEntropyLoss()
criterion.cuda()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

checkpoint = torch.load(opt.load_model_path)
net.load_state_dict(checkpoint['model_state_dict'])  


if torch.cuda.device_count() > 1:
    net = torch.nn.DataParallel(net)
net.cuda()

print("done")

done


In [8]:
###########################################
# test with CNN model + Joint Bayesian
###########################################
import os
import torchvision.transforms as transforms
from PIL import Image
import numpy as np


normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)

def verificationTransform(imagefile):
    veri_transform = transforms.Compose([
                transforms.Scale(299),
                transforms.CenterCrop(299), 
                transforms.ToTensor(),
                normalize
        ]) 
    
    imagefile = os.path.join(opt.path, imagefile)
    image = Image.open(imagefile)
    image = veri_transform(image)

    return image.unsqueeze_(0)


level = 'easy'  # or 'hard' or 'medium'
DATA_DIR = '../data/compcars/data/'
test_file = os.path.join(DATA_DIR, 'train_test_split/verification/verification_pairs_%s.txt' % level)
test_list = list(map(lambda s: s.strip().split(' '), open(test_file).readlines()))
N = len(test_list)
preds = np.zeros((N,))
gts = np.array(map(lambda t: int(t[-1]), test_list))
hit_num = 0


inFCVeri = []
outFCVeri = []
def hookVerification(module, i, o):
    inFCVeri.append(i)
    outFCVeri.append(o)
    
    
model = net  
model.eval()
model.module.fc.register_forward_hook(hookVerification)
featureList = []
currentFeature = []

for i, test_data in enumerate(test_list):
    gt = int(test_data[-1])
    t1 = time.time()
    
    # 1/ fetch image file
    imgfile1, imgfile2 = test_data[0], test_data[1]
    img1 = verificationTransform(imgfile1)
    img2 = verificationTransform(imgfile2)
    img1, img2 = Variable(img1.cuda()), Variable(img2.cuda())

    # 2/ fetch feature matrixs
    x1 = model(img1)
    x1_in2048 = inFCVeri[0][0][0].data.cpu().numpy()
    inFCVeri = []
    outFCVeri = []
    
    x2 = model(img2)
    x2_in2048 = inFCVeri[0][0][0].data.cpu().numpy()
    inFCVeri = []
    outFCVeri = []
    # x1 shape torch.Size([2048]), x2 shape torch.Size([2048])
     
    # 3/ PCA to 20 
    xx1 = pca_model.transform(np.asmatrix(x1_in2048))
    xx2 = pca_model.transform(np.asmatrix(x2_in2048))
    #pca_model.components_.reshape(2048, 20).shape
    # shapes (1,40960) and (20,20) not aligned: 40960 (dim 1) != 20 (dim 0)    

    # 4/ JB verification    
    t2 = time.time()
    preds[i] = jbverify(A, G, xx1, xx2)    
    #ratio=np.dot(np.dot(np.transpose(x1),A),x1)+np.dot(np.dot(np.transpose(x2),A),x2)-2*np.dot(np.dot(np.transpose(x1),G),x2)
    # A (20, 20), G(20, 20)
    
    t3 = time.time() 
    if (preds[i]>opt.jointBayesianThreshold and gt==1) or(preds[i]<opt.jointBayesianThreshold and gt==0):
        hit_num = hit_num+1

    if (i+1)%1000==0:
        print('***************** image %d *******************' % (i+1))
        print('%d --- Feature extraction in %f s, prediction: %f s\nresult: %f(%d)' % (i+1, t2-t1, t3-t2, preds[i], gt))
    
    

    
test_list2 = list(map(lambda s: s.strip().split(' '), open(test_file).readlines()))
gts = np.asarray(list(map(lambda t: int(t[-1]), test_list2)))
  
print('pos mean, std, mean, max : %f, %f, %f, %f' % (preds[gts==1].mean(), preds[gts==1].std(), preds[gts==1].min(), preds[gts==1].max()))
print('neg mean, std, mean, max : %f, %f, %f, %f' % (preds[gts==0].mean(), preds[gts==0].std(), preds[gts==0].min(), preds[gts==0].max()))
print('hit rate: %f' % (hit_num*1.0/N))
  
    

# Dec 2 2018
# preds 1 mean -3.9081270492731113 std 11.807556093672183 min -79.43003772026832 max 44.3816863572722
# preds 0 mean -37.01217310158124 std 21.332399250284972 min -128.55467633653916 max 21.060876005203426

# Dec 3 2018
# pos mean, std, mean, max : -3.908126, 11.807555, -79.430037, 44.381689
# neg mean, std, mean, max : -37.012171, 21.332399, -128.554675, 21.060877



***************** image 1000 *******************
1000 --- Feature extraction in 0.202955 s, prediction: 0.000050 s
result: -1.661495(1)
***************** image 2000 *******************
2000 --- Feature extraction in 0.208756 s, prediction: 0.000059 s
result: -8.930168(1)
***************** image 3000 *******************
3000 --- Feature extraction in 0.231494 s, prediction: 0.000058 s
result: -16.310215(1)
***************** image 4000 *******************
4000 --- Feature extraction in 0.251695 s, prediction: 0.000059 s
result: 2.756601(1)
***************** image 5000 *******************
5000 --- Feature extraction in 0.178049 s, prediction: 0.000059 s
result: -5.391550(1)
***************** image 6000 *******************
6000 --- Feature extraction in 0.112487 s, prediction: 0.000051 s
result: -6.467470(1)
***************** image 7000 *******************
7000 --- Feature extraction in 0.211112 s, prediction: 0.000048 s
result: 3.033164(1)
***************** image 8000 *******************
